### Import Libraries

In [1]:
import numpy as np 
from std_msgs.msg import Header
from trajectory_msgs.msg import JointTrajectory
from trajectory_msgs.msg import JointTrajectoryPoint
import rospy
from sensor_msgs.msg import PointCloud2
import sensor_msgs.point_cloud2 as pc2
import pcl
import pcl_helper
from pcl_helper import *
import tf
import sys
import rospkg
import moveit_commander
import tf
import os 
import numpy as np 
import copy
import time

from geometry_msgs.msg import (
    PoseStamped,
    Pose,
    Point,
    Quaternion,
)
from gazebo_msgs.srv import (SpawnModel)
from geometry_msgs.msg import (PoseStamped,Pose,Point,Quaternion)

ModuleNotFoundError: No module named 'moveit_commander'

In [3]:
# 복셀화(Down sampling)
def do_voxel_grid_downssampling(pcl_data,leaf_size):
    vox = pcl_data.make_voxel_grid_filter()
    vox.set_leaf_size(leaf_size, leaf_size, leaf_size)
    return  vox.filter()

# 노이즈 제거
def do_statistical_outlier_filtering(pcl_data,mean_k,tresh):
    outlier_filter = pcl_data.make_statistical_outlier_filter()
    outlier_filter.set_mean_k(mean_k)
    outlier_filter.set_std_dev_mul_thresh(tresh)
    return outlier_filter.filter()

# 바닥 제거 함수(random sample consensus 이용)
def do_ransac_plane_segmentation(pcl_data,pcl_sac_model_plane,pcl_sac_ransac,max_distance):
    '''
    Create the segmentation object
    :param pcl_data: point could data subscriber
    :param pcl_sac_model_plane: use to determine plane models, pcl.SACMODEL_PLANE
    :param pcl_sac_ransac: RANdom SAmple Consensus, pcl.SAC_RANSAC
    :param max_distance: Max distance for apoint to be considered fitting the model, 0.01
    :return: segmentation object
    '''
    seg = pcl_data.make_segmenter()
    seg.set_model_type(pcl_sac_model_plane)
    seg.set_method_type(pcl_sac_ransac)
    seg.set_distance_threshold(max_distance)

    # outliner 추출
    inliners, _ = seg.segment()
    inliner_object = pcl_data.extract(inliners,negative=False)
    outliner_object = pcl_data.extract(inliners,negative=True)
    return outliner_object

# clustering 함수
def euclid_cluster(cloud):
    white_cloud = XYZRGB_to_XYZ(cloud)
    tree = white_cloud.make_kdtree()
    ec = white_cloud.make_EuclideanClusterExtraction()
    ec.set_ClusterTolerance(0.015)
    ec.set_MinClusterSize(20)
    ec.set_MaxClusterSize(3000)
    ec.set_SearchMethod(tree)
    cluster_indices = ec.Extract()
    return cluster_indices, white_cloud

def cluster_mask(cluster_indices, white_cloud):
    cluster_color = get_color_list(len(cluster_indices))
    color_cluster_point_list = []
    for j, indices in enumerate(cluster_indices):
        for i, indice in enumerate(indices):
            color_cluster_point_list.append([
                                            white_cloud[indice][0],
                                            white_cloud[indice][1],
                                            white_cloud[indice][2],
                                            rgb_to_float( cluster_color[j] )
                                           ])
    cluster_cloud = pcl.PointCloud_PointXYZRGB()
    cluster_cloud.from_list(color_cluster_point_list)
    return cluster_cloud

# 원하는 object의 data만 추출하는 함수
def choose_obj_group(cluster_indices,white_cloud):
    size = white_cloud.size
    obj = max(white_cloud)
    for i in range(size):
        if white_cloud[i] == obj:
            obj_index = i
    for j,indices in enumerate(cluster_indices):
            for k,indice in enumerate(indices):
                if indice == obj_index:
                    cluster_index = j
    obj_cluster = cluster_indices[cluster_index]
    obj_point_list = []
    for i,indice in enumerate(obj_cluster):
        obj_point_list.append([
                    white_cloud[indice][0],
                    white_cloud[indice][1],
                    white_cloud[indice][2],
                    1.5
                ])
    obj_cloud = pcl.PointCloud_PointXYZRGB()
    obj_cloud.from_list(obj_point_list)
    return obj_cloud

# pcd data를 받아 중심점을 return하는 함수
def get_middle_point(pcd):
    x_total = 0
    y_total = 0
    z_total = []
    pcd_numpy = pcd.to_array()
    for i in range(len(pcd_numpy)):
        x_total += pcd_numpy[i,0]
        y_total += pcd_numpy[i,1]
        z_total.append(pcd_numpy[i,2])
    x = x_total / len(pcd_numpy)
    y = y_total / len(pcd_numpy)
    z = (max(z_total)+(min(z_total)))/2
    return x,y,z

# 관심 영역 설정
def do_passthrough(pcl_data,filter_axis,axis_min,axis_max):
    '''
    Create a PassThrough  object and assigns a filter axis and range.
    :param pcl_data: point could data subscriber
    :param filter_axis: filter axis
    :param axis_min: Minimum  axis to the passthrough filter object
    :param axis_max: Maximum axis to the passthrough filter object
    :return: passthrough on point cloud
    '''
    passthrough = pcl_data.make_passthrough_filter()
    passthrough.set_filter_field_name(filter_axis)
    passthrough.set_filter_limits(axis_min, axis_max)
    return passthrough.filter()

# 좌표 변환 함수
def tf_matrix():
    listener = tf.TransformListener() 
    listener.waitForTransform('world','camera_depth_optical_frame',rospy.Time(),rospy.Duration(2))
    (t,q) = listener.lookupTransform('world','camera_depth_optical_frame', rospy.Time(0))
    t_matrix = tf.transformations.translation_matrix(t)
    r_matrix = tf.transformations.quaternion_matrix(q)
    return np.dot(t_matrix,r_matrix)

def change_frame(matt, points):
    transpose = points[:,0:3]
    ones = np.ones((len(points),1))
    transpose = np.concatenate((transpose,ones),axis=1)
    transpose = transpose.T
    transpose_after = np.dot(matt,transpose)
    transpose_after = transpose_after.T
    transpose_after_after = transpose_after[:,0:3]
    rgb = points[:,3].reshape(len(points),1)
    finalmat = np.concatenate((transpose_after_after,rgb),axis=1)
    return finalmat

# numpy에서 pcd data로 바꾸는 함수
def numpy_to_pcd(nump):
    nump = nump.astype(np.float32)
    pcd = pcl.PointCloud_PointXYZI()
    pcd.from_array(nump)
    return pcd

# callback 함수
def callback(input_ros_msg):
    cloud = pcl_helper.ros_to_pcl(input_ros_msg)
    cloud = do_voxel_grid_downssampling(cloud,0.005)
    delete_floor = do_ransac_plane_segmentation(cloud,pcl.SACMODEL_PLANE,pcl.SAC_RANSAC,0.01)
    delete_desk = do_ransac_plane_segmentation(delete_floor,pcl.SACMODEL_PLANE,pcl.SAC_RANSAC,0.01)

    delete_desk_1 = delete_desk.to_array()
    A = tf_matrix()
    delete_desk_points = change_frame(A,delete_desk_1)
    delete_desk_2 = numpy_to_pcd(delete_desk_points)
    cluster_indices, white_cloud = euclid_cluster(delete_desk_2)

    # cluster된 물체들 전부
    get_color_list.color_list = []
    final = cluster_mask(cluster_indices,white_cloud)
    final = do_passthrough(final,'z',0.75,1.2)
    final_new = pcl_helper.pcl_to_ros(final)
    # 원하는 object만 추출
    obj_cloud = choose_obj_group(cluster_indices,white_cloud)
    # pcl.save(obj_cloud,"obj_cloud2.pcd")
    obj_new = pcl_helper.pcl_to_ros(obj_cloud)
    pub.publish(final_new)

In [3]:
rospy.init_node('publish_joints')
arm_joint_names = ['shoulder_pan_joint','shoulder_lift_joint','elbow_joint', 
                'wrist_1_joint', 'wrist_2_joint','wrist_3_joint']
gripper_joint_names = ['gripper_finger1_joint']
rate    = rospy.Rate(1)
arm_pub = rospy.Publisher('/arm_controller/command', JointTrajectory, queue_size=10)
gripper_pub = rospy.Publisher('/gripper_controller/command', JointTrajectory, queue_size=10)

def move_gripper(joint):
    global gripper_joint_names, gripper_pub
    """ Open- Close Gripper """
    gripper        = JointTrajectory()
    gripper_value  = JointTrajectoryPoint()
    gripper.header = Header()
    gripper.joint_names = gripper_joint_names
    gripper_value.positions = [joint] # Open pose
    gripper_value.time_from_start = rospy.Duration.from_sec(0.01)
    gripper.points.append(gripper_value)
    gripper_pub.publish(gripper)


def move_arm(joint_seq): 
    Flag = True 
    global arm_joint_names, rate, arm_pub
    while Flag:
        """ Move to pick an object """
        for idx, joints in enumerate(joint_seq):
            joint = joints.reshape([6,])
            arm = JointTrajectory()
            arm_value = JointTrajectoryPoint()
            arm.header = Header()
            arm.joint_names= arm_joint_names
            arm_value.positions       = joint
            arm_value.time_from_start = rospy.Duration.from_sec(0.01)
            arm.points.append(arm_value)
            arm_pub.publish(arm)
            rate.sleep()    
            if idx+1 == len(joint_seq): 
                Flag = False 

In [4]:
""" Test """
joint_seq  = np.array([[2.0, -0.6596, 1.3364, 0.0350, 0, 0]]) 
move_arm(joint_seq)
move_gripper(0.8) # Open 
# move_gripper(0.3) # Close
print("Publish trajectories")

Publish trajectories


In [8]:
def spawn_gazebo_model(model_path, model_name, model_pose, reference_frame="world"):
  """
  Spawn model in gazebo
  """
  model_xml = ''
  with open(model_path, "r") as model_file:
    model_xml = model_file.read().replace('\n', '')
  rospy.wait_for_service('/gazebo/spawn_sdf_model')
  try:
    spawn_sdf = rospy.ServiceProxy('/gazebo/spawn_sdf_model', SpawnModel)
    resp_sdf= spawn_sdf(model_name, model_xml, "/", model_pose, reference_frame)
  except rospy.ServiceException:
    rospy.logerr("Spawn SDF service call failed")
moveit_commander.roscpp_initialize(sys.argv)
robot         = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
# Parameters 
table_height    = 0.79
# Cylinder 
cylinder_height = 0.2
cylinder_radius = 0.03
cylinder_pos_x  = 0.97 
cylinder_pos_y  = 0 
cylinder_pos_z  = table_height+cylinder_height/2.
# Box 
box_height      = 0.06
box_x           = 0.06 
box_y           = 0.06
box_pos_x       = 0.97 
box_pos_y       = 0 
box_pos_z       = table_height+box_height/2.
""" RViz """
# Cylinder 
cylinder_name = "cylinder"
cylinder_pos  = PoseStamped()
cylinder_pos.header.frame_id    = robot.get_planning_frame()
cylinder_pos.pose.position.x    = cylinder_pos_x
cylinder_pos.pose.position.y    = cylinder_pos_y
cylinder_pos.pose.position.z    = cylinder_pos_z
cylinder_pos.pose.orientation.w = 1.0  
# Box 
box_name = "box"
box_pos  = PoseStamped()
box_pos.header.frame_id    = robot.get_planning_frame()
box_pos.pose.position.x    = box_pos_x
box_pos.pose.position.y    = box_pos_y
box_pos.pose.position.z    = box_pos_z
box_pos.pose.orientation.w = 1.0  

""" GAZEBO """
rospack = rospkg.RosPack()
pack_path = rospack.get_path('ir_gazebo')
object_path = pack_path+os.sep+'urdf'+os.sep+'object'+os.sep+'cylinder.sdf'
object_name = cylinder_name
object_pose = Pose(position=Point(x=cylinder_pos_x , y=cylinder_pos_y, z=cylinder_pos_z))

In [9]:
spawn_gazebo_model(object_path, object_name, object_pose)

In [5]:
rospy.init_node('pointcloud', anonymous=True)
rospy.Subscriber('/camera/depth/color/points', PointCloud2, callback)
pub = rospy.Publisher("/camera/depth/color/points_new",PointCloud2,queue_size=1)
rospy.spin()

: 

: 